<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/YouTubeToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U openai-whisper

In [ ]:
pip install pytubefix

In [ ]:
pip install moviepy

In [ ]:
pip install google-api-python-client

In [7]:
import html
import os
import datetime
import csv
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
from pytubefix import YouTube
from moviepy.editor import *
import whisper

#-------↓パラメータ入力↓-------

developerKey = ""
channel_id = "UCmuo6-sGvNxqT2NF7SWKClg"

#-------↑パラメータ入力↑-------

youtube = build("youtube", "v3", developerKey=developerKey)

videos = [] #videoURLを格納する配列
dt_now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

def youtube_search(pagetoken, st, ed):

    youtube = build("youtube", "v3", developerKey=developerKey)

    search_response  = youtube.search().list(
        part = "snippet",
        channelId = channel_id,
        maxResults = 50,
        type = "video",
        publishedAfter = st, #'2013-01-01T00:00:00Z',
        publishedBefore = ed, #'2014-01-01T00:00:00Z',
        order = "date",
        pageToken = pagetoken
    ).execute()

    print(search_response["pageInfo"]["totalResults"])
    for search_result in search_response.get("items", []):

        if search_result["id"]["kind"] == "youtube#video":
            publishedAt = search_result["snippet"]["publishedAt"]
            title = html.unescape(search_result["snippet"]["title"])
            id = search_result["id"]["videoId"]
            url = 'https://www.youtube.com/watch?v=%s' % search_result["id"]["videoId"]

            videos.append([publishedAt, title, id, url])
            youtube_to_text(id, url)
    try:
        nextPagetoken = search_response["nextPageToken"]
        youtube_search(nextPagetoken, st, ed)
    except:
        return

def youtube_to_text(id, url):

    # YouTube動画をロード
    yt = YouTube(url)

    # 最高品質のオーディオストリームを選択
    audio_stream = yt.streams.filter(only_audio=True).first()

    # オーディオを一時ファイルとしてダウンロード
    temp_file = audio_stream.download()

    # 出力ファイル名
    audiofile = id + ".mp3"

    # MoviePyを使用してオーディオをMP3に変換
    audio_clip = AudioFileClip(temp_file)
    audio_clip.write_audiofile("/content/drive/MyDrive/" + audiofile, codec="libmp3lame")

    # 一時ファイルを削除
    os.remove(temp_file)

    # 出力ファイル名（適宜変更する）
    outputfile = id + ".txt"

    # 処理
    # model = whisper.load_model("small", device="cpu")
    model = whisper.load_model("medium", device="cuda")
    result = model.transcribe("/content/drive/MyDrive/" + audiofile, language="ja", verbose=True)

    # セグメントごとに改行してテキストを取得
    segments = result["segments"]
    transcript = "\n".join(segment["text"] for segment in segments)

    # txtへ書き出し
    with open("/content/drive/MyDrive/" + outputfile, "w", encoding='utf-8_sig') as f:
        f.write(transcript)

# 2023年1月1日から（ここを適宜変更）
dt = datetime.datetime(2023, 1, 1, 0, 0)
#
for i in range(1, 21):
    youtube_search('', dt.isoformat()+'Z', (dt + relativedelta(months=1)).isoformat()+'Z')
    dt = dt + relativedelta(months=1)

31
MoviePy - Writing audio in /content/drive/MyDrive/MTilWTLmVUU.mp3


MoviePy - Done.


100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 104MiB/s]
  checkpoint = torch.load(fp, map_location=device)



[00:00.000 --> 00:04.880] 今回もポケットモンスターソードシールドについての考察をやっていこうと思うんですけども
[00:04.880 --> 00:09.440] ダクマやバドレックス、心霊時系ポケモンたちが話題になってるんですけども
[00:09.440 --> 00:15.200] 実はそのかげでもしかしたら登場するのではないかと考えられているポケモンたちがいるんです
[00:15.200 --> 00:17.440] それがEVフレンズ
[00:17.440 --> 00:21.280] 今回はポケットモンスターキンギンダイヤモンドパールXY
[00:21.280 --> 00:23.840] そしてダウンロードコンテンツが示す意味
[00:23.840 --> 00:27.360] 引いてはリストラ問題を軸に考察していこうと思います
[00:27.520 --> 00:32.000] 実は新たなEVフレンズのタイプが判明してくるんです
[00:32.000 --> 00:34.720] ということで今回は新たなEVフレンズが
[00:34.720 --> 00:40.640] 鎧のことを冠の節弦に登場する可能性について徹底考察したいと思います
[00:40.640 --> 00:41.920] それではどうぞ
[00:41.920 --> 00:43.280] EVとVス
[00:43.280 --> 00:48.080] まずは今回の動画の主役EVについて簡単に紹介したいと思います
[00:48.080 --> 00:49.840] 図鑑ナンバー133
[00:49.840 --> 00:50.960] 名前EV
[00:50.960 --> 00:52.160] 英語名EV
[00:52.160 --> 00:53.680] 分類進化ポケモン
[00:53.680 --> 00:54.720] タイプノーマル
[00:54.720 --> 00:56.400] 高さ0.3メートル
[00:56.400 --> 00:57.760] 重さ6.5キロ
[00:57.760 --> 01:03.440] 名前の例は進化を意味する英語エボリューションの頭文字のEとVから来てるとのことです
[01:03.440 --> 01:10.480

MoviePy - Done.


...

[00:00.000 --> 00:05.340] 十二鬼月の中でも、回6名の鬼。その名は、カゲン。
[00:05.340 --> 00:09.800] カゲンたちは十二鬼月である以上、数百人の人を喰らい、
[00:09.800 --> 00:14.200] 文字通り、波の気殺対しては歯が立たない強さを持っていることから、
[00:14.200 --> 00:16.800] 凶悪な鬼として名を馳せていました。
[00:16.800 --> 00:24.260] こうした強さから、鬼にとっては、カゲンであっても十二鬼月に選ばれることは、史上の名誉なんです。
[00:24.260 --> 00:30.200] 現在クライマックスを迎えている、鬼滅の刃においても、作中ではカゲンの鬼たちが存在しており、
[00:30.200 --> 00:35.560] 彼らは主人公、かまどたんじろうの前に立ちあたかり、苦しめた凶悪な鬼たちでした。
[00:35.560 --> 00:41.760] そんな中で、鬼滅史上最悪のパワハラ事件が起きてしまうんです。
[00:41.760 --> 00:46.900] ということで、今回は十二鬼月のカゲンの鬼たちについて徹底解説。
[00:46.900 --> 00:53.400] そして本誌では語られていない、知られざるカゲンのエピソードを完全解説したいと思います。
[00:53.400 --> 00:58.700] なお、ジャンプ最新後198話までのネタバレを含みますので、ご注意ください。
[00:58.700 --> 01:05.780] 今回徹底解説会となってますので、いつものように画面に全集中、常中でよろしくお願いします。
[01:05.780 --> 01:07.280] それでは、どうぞ。
[01:09.280 --> 01:14.000] 赤座のように顔にイレズミがあり、髪を後ろで縛っている鬼カマヌエ。
[01:14.000 --> 01:21.560] 元々十二鬼月の末席、カゲンの六を預かっていた境外に代わり、昇格したのがカマヌエだと思われます。
[01:21.560 --> 01:27.480] そんなカマヌエは第51話にて、ルイが殺されたことでイカルムザンに無限上に召集され、
[01:27.480 --> 01:30.980

MoviePy - Done.


  checkpoint = torch.load(fp, map_location=device)



[00:00.000 --> 00:05.600] 連国家にある変柱の書によると、その呼吸を極めた者は超屋敷の人々から、
[00:05.600 --> 00:11.300] ひいては女性大使から嫌われると伝承された呼吸法がある。その名も。
[00:11.300 --> 00:16.800] ということで今回は鬼滅のエヴァの人気キャラクター投票を開催したいと思います。
[00:16.800 --> 00:18.100] テーマはこちらです。
[00:18.100 --> 00:23.600] みんなが決める可愛いキャラクターランキング、最強の美女、美少女は誰だ?
[00:23.600 --> 00:28.800] そしてかっこいいキャラクターランキング、最強のイケメン、美男子は誰だ?
[00:28.900 --> 00:33.000] フィーナルエントリーはというと、柱屋、五感組や鬼はもちろん、
[00:33.000 --> 00:37.200] サビトや村田さんまでほぼ全てのキャラクターを網羅しています。
[00:37.200 --> 00:41.000] 可愛いランキングにおいても柱屋、五感組や鬼はもちろん、
[00:41.000 --> 00:45.900] 神崎、葵、うずいの妻、より一の妻、うたや澄よし、すや子など、
[00:45.900 --> 00:48.900] ほぼ全てのキャラクターを網羅しています。
[00:48.900 --> 00:52.500] ぜひともどしどし投票の方をよろしくお願いします。
[00:52.500 --> 00:57.000] あくまでキャラを区別や差別するためのものではございませんのでご了承ください。
[00:57.000 --> 01:03.500] 投票方法は概要欄にURLがありますので、URLの先のアンケートフォームから投票可能になっています。
[01:03.500 --> 01:06.600] 簡単に投票できるのでぜひともご参加ください。
[01:06.600 --> 01:10.400] ちなみにアンケートフォームにないキャラクターについてはお手数なんですけども、
[01:10.400 --> 01:16.400] アンケートフォームの中にその他という欄があるので、そこに記載の上で投票の方をよろしくお願いします。


OSError: [Errno 36] File name too long: '/content/【鬼滅の刃】鬼の最高位｢十二鬼月｣!!”最強”12体の鬼に隠された過去と家族愛&知られざる感動のエピソードを徹底解説【上弦ノ鬼】【下弦ノ鬼】【黒死牟】【累】【童磨】【猗窩座】【きめつのやいば】.mp4'

In [12]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/drive/MyDrive/download.zip /content/drive/MyDrive

  adding: content/drive/MyDrive/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled0.ipynb (deflated 58%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled1 (1).ipynb (deflated 87%)
  adding: content/drive/MyDrive/Colab Notebooks/Whisper YouTube in SRT.ipynb のコピー (deflated 60%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled2.ipynb (deflated 30%)
  adding: content/drive/MyDrive/Colab Notebooks/Whisper.ipynb のコピー (deflated 67%)
  adding: content/drive/MyDrive/Colab Notebooks/Untitled1.ipynb (deflated 66%)
  adding: content/drive/MyDrive/Colab Notebooks/WhisperOnGoogleColab.ipynb (deflated 66%)
  adding: content/drive/MyDrive/Colab Notebooks/YouTubeToText.ipynb (deflated 73%)
  adding: content/drive/MyDrive/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/MTilWTLmVUU.mp3 (deflated 1%)
  adding: content/drive/MyDrive/MTilWTLmVUU.txt (deflated 67%)
  adding: content/drive/MyDrive

In [14]:
# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/drive/MyDrive/download.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>